In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
from zipfile import ZipFile
from PIL import Image
import sys
import os
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt

In [3]:
import torch
from torchvision import transforms, datasets

In [4]:
data_transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        # transforms.RandomSizedCrop(224),
#         transforms.RandomHorizontalFlip(),
#         transforms.RandomVerticalFlip(),
#         transforms.RandomRotation(degrees=180),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

train_fer = datasets.ImageFolder(root='/tf/data/Quan/fer2013/data/train', transform=data_transform)
val_fer = datasets.ImageFolder(root='/tf/data/Quan/fer2013/data/val', transform=data_transform)
test_fer = datasets.ImageFolder(root='/tf/data/Quan/fer2013/data/test', transform=data_transform)

train_loader = torch.utils.data.DataLoader(train_fer,
                                             batch_size=128, shuffle=True,
                                             num_workers=8)
val_loader = torch.utils.data.DataLoader(val_fer,
                                             batch_size=128, shuffle=True,
                                             num_workers=8)
test_loader = torch.utils.data.DataLoader(test_fer,
                                             batch_size=128, shuffle=True,
                                             num_workers=8)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn import metrics

In [6]:
class SVblock(nn.Module):
    def __init__(self, in_neurons, nb_neurons, kernel_size=(3,3), batch_norm=True, activation='relu', **kwargs):
        '''
        in_neurons: C_in
        nb_neurons: n_filters of conv2d
        '''
        super(SVblock, self).__init__()

        self.index = kwargs['index'] if 'index' in kwargs else None
        self.in_neurons = in_neurons
        self.nb_neurons = nb_neurons
        self.conv1 = nn.Conv2d(self.in_neurons, self.nb_neurons, kernel_size=kernel_size, padding=1, stride=1)
        self.conv2 = nn.Conv2d(self.nb_neurons, self.nb_neurons, kernel_size=kernel_size, padding=1, stride=1)
        self.batchnorm1 = nn.BatchNorm2d(nb_neurons)
        self.batchnorm2 = nn.BatchNorm2d(nb_neurons)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=(2,2), stride=2)

    def forward(self, x):
        '''
        x: single output layer of list 
        '''

        x = torch.cat(x, dim=1) if type(x) is list else x
        # print('x in svblock: ', x.is_cuda)

        x = x.to(device)

        # 1st
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)


        # #2nd
        # x = self.conv2(x)
        # x = self.batchnorm(x)
        # x = self.relu(x)

        # x = self.conv2(x)
        # x = self.batchnorm(x)
        # x = self.relu(x)

        # maxpooling
        x = self.maxpool(x)

        return x

In [7]:
class DownSampling(nn.Module):
    def __init__(self, in_size, out_size):
        '''
        in_size: input size (H_in x W_in)
        out_size: output size (H_out x W_out)
        '''

        super(DownSampling, self).__init__()

        self.in_size = in_size
        self.out_size = out_size

        h_crop = (in_size[0] - out_size[0])
        w_crop = (in_size[1] - out_size[1])

        self.h_top = h_crop // 2
        self.h_bottom = h_crop // 2 if not h_crop % 2 else (h_crop // 2) + 1

        self.w_left = w_crop // 2
        self.w_right = w_crop // 2 if not w_crop % 2 else (w_crop // 2) + 1
        
    def forward(self, x):
        return x[:,:, self.h_top : (x.size(-2) - self.h_bottom) , self.w_left : (x.size(-1) - self.w_right)]


In [8]:
class DownIncreaseDepth(nn.Module):
    def __init__(self, out_size, in_depth, out_depth):
        super(DownIncreaseDepth, self).__init__()
        
        self.out_size = out_size
        self.in_depth = in_depth
        self.out_depth = out_depth
        
        self.iconv = nn.Conv2d(self.in_depth, self.out_depth, kernel_size=1, padding=0, stride=1)
        self.relu = nn.ReLU()
        self.batchnorm = nn.BatchNorm2d(self.out_depth)
        
    def forward(self, x):
        x = F.interpolate(x, self.out_size)
        x = self.iconv(x)
        x = self.relu(x)
        x - self.batchnorm(x)
        
        return x
        

In [14]:
layer = DownIncreaseDepth((16,16), 64, 256)

t = layer(torch.rand(64, 64, 48, 48))
t.size()

torch.Size([64, 256, 16, 16])

In [12]:
F.interpolate(torch.rand(100, 3, 48,48), (24,24))

tensor([[[[5.2505e-01, 6.9117e-02, 9.4545e-01,  ..., 3.7437e-01,
           5.6068e-01, 9.7229e-01],
          [2.4384e-01, 5.8731e-01, 5.8246e-01,  ..., 4.4137e-02,
           7.4270e-01, 9.9532e-02],
          [3.5889e-01, 3.5253e-01, 8.9325e-01,  ..., 9.6012e-01,
           5.2550e-01, 8.3826e-01],
          ...,
          [7.0460e-01, 2.0373e-01, 6.7805e-01,  ..., 1.4819e-01,
           5.8890e-01, 4.7762e-01],
          [2.7905e-01, 4.6535e-01, 7.3626e-01,  ..., 3.2057e-02,
           9.8646e-01, 8.6765e-01],
          [6.9949e-01, 6.7878e-02, 5.8984e-02,  ..., 8.1515e-01,
           7.1082e-01, 8.0648e-01]],

         [[9.2451e-01, 3.5965e-01, 6.4952e-01,  ..., 6.2631e-01,
           5.3915e-02, 1.9383e-01],
          [3.1127e-01, 6.3249e-01, 9.4133e-01,  ..., 2.6083e-01,
           3.7129e-01, 2.4450e-01],
          [5.1830e-01, 3.5714e-01, 1.2137e-01,  ..., 8.1343e-01,
           2.4743e-01, 5.7108e-01],
          ...,
          [9.9282e-01, 5.7083e-02, 7.5685e-01,  ..., 8.4810

In [11]:
class SkippedVGG(nn.Module):
    def __init__(self, in_size, nb_blocks, n_classes, **kwargs):
        '''
        in_size (without batch size): (C, H, W)

        '''
        super(SkippedVGG, self).__init__()

        # init power of filters
        init_pf = kwargs['init_pf'] if 'init_pf' in kwargs else 6
        self.device = torch.device('cpu') if not 'device' in kwargs else kwargs['device']

        self.nb_blocks = nb_blocks
        self.n_classes = n_classes

        self.ln = nn.Linear(1920, 512)
        self.batchnorm = nn.BatchNorm1d(512)
        self.relu = nn.ReLU()
        self.ln2 = nn.Linear(512, self.n_classes)
        self.batchnorm2 = nn.BatchNorm1d(7)

        self.init_conv = nn.Conv2d(in_size[0], 64, kernel_size=(3,3), padding=1, stride=1)
        self.sv_blocks = []

        # init crop layers
        self.crop_layers = []
        for i in range(self.nb_blocks):
            self.crop_layers.append([])

        cur_size = (in_size[1], in_size[2])

        out_last_block = 0
        for i in range(self.nb_blocks):
            if i == 0:
                self.sv_blocks.append(SVblock(in_size[0], 2**(init_pf)).to(self.device))
                out_last_block += 2**(init_pf)
            
            else:
                # self.sv_blocks.append(SVblock(2**(init_pf+i-1), 2**(init_pf+i)))
                self.sv_blocks.append(SVblock(out_last_block, 2**(init_pf+i)).to(self.device))
                out_last_block += 2**(init_pf+i)
        
            out_block_size = (cur_size[0] // 2, cur_size[1] // 2)
            
            for k, j in enumerate(range(i+1, self.nb_blocks)):
                target_size = (out_block_size[0] // (2**(k+1)), out_block_size[1] // (2**(k+1)))

                crop_layer = DownSampling(out_block_size, target_size).to(self.device)
#                 crop_layer = DownIncreaseDepth(target_size, 2**(init_pf+i), out_last_block + (2**(init_pf+k))).to(self.device)

                self.crop_layers[j].append(crop_layer.to(self.device))
                
            cur_size = out_block_size
  
    def forward(self, x):
        outs = []
        for i in range(self.nb_blocks):
            
            _crops = self.crop_layers[i-1] if i >=1 else []
            
            for j, clayer in enumerate(_crops):
                # print(clayer(x).size())
                x = torch.cat((x, clayer(outs[j])), dim=1)

            x = self.sv_blocks[i](x)
            outs.append(x)

        _crops = self.crop_layers[-1]
        for j, clayer in enumerate(_crops):
            # print(j)
            # print(x.size())
            # print(outs[j].size())
            # print(clayer)
            x = torch.cat((x, clayer(outs[j])), dim=1)

        x = nn.AvgPool2d(kernel_size=(x.size(-2), x.size(-1)))(x)
        x = nn.Flatten()(x)
#         print(x.size())
        x = self.ln(x)
        x = self.batchnorm(x)
        # x = self.relu(x)
        x = self.ln2(x)
        x = self.batchnorm2(x)

        return x


In [10]:
class SkippedVGG(nn.Module):
    def __init__(self, in_size, nb_blocks, n_classes, **kwargs):
        '''
        in_size (without batch size): (C, H, W)

        '''
        super(SkippedVGG, self).__init__()

        # init power of filters
        init_pf = kwargs['init_pf'] if 'init_pf' in kwargs else 6
        self.device = torch.device('cpu') if not 'device' in kwargs else kwargs['device']

        self.nb_blocks = nb_blocks
        self.n_classes = n_classes

        self.ln = nn.Linear(1920, 512)
        self.batchnorm = nn.BatchNorm1d(512)
        self.relu = nn.ReLU()
        self.ln2 = nn.Linear(512, self.n_classes)
        self.batchnorm2 = nn.BatchNorm1d(7)

        self.init_conv = nn.Conv2d(in_size[0], 64, kernel_size=(3,3), padding=1, stride=1)
        self.sv_blocks = []

        # init crop layers
        self.crop_layers = []
        for i in range(self.nb_blocks):
            self.crop_layers.append([])

        

        out_last_block = 0
        for i in range(self.nb_blocks):
            if i == 0:
                self.sv_blocks.append(SVblock(in_size[0], 2**(init_pf)).to(self.device))
                out_last_block += 2**(init_pf)
            
            else:
                # self.sv_blocks.append(SVblock(2**(init_pf+i-1), 2**(init_pf+i)))
                self.sv_blocks.append(SVblock(out_last_block, 2**(init_pf+i)).to(self.device))
                out_last_block += 2**(init_pf+i)
                
        
        cur_size = (in_size[1], in_size[2])
        
        for i in range(self.nb_blocks):
            out_block_size = (cur_size[0] // 2, cur_size[1] // 2)
            
            for k, j in enumerate(range(i, self.nb_blocks)):
                target_size = (out_block_size[0] // (2**(k+1)), out_block_size[1] // (2**(k+1)))
                
                crop_layer = DownIncreaseDepth(target_size, self.sv_blocks[i+1].nb_neurons, self.sv_blocks[j].in_neurons).to(self.device)
                
                print('Block ', i+1, ': ', self.sv_blocks[i].nb_neurons, ' -- ', self.sv_blocks[j].in_neurons, ' down to block ', j, ': ', target_size)

                self.crop_layers[j].append(crop_layer.to(self.device))
                
            cur_size = out_block_size
  
    def forward(self, x):
        outs = []
        for i in range(self.nb_blocks):
            
            _crops = self.crop_layers[i-1] if i >=1 else []
            
            for j, clayer in enumerate(_crops):
                # print(clayer(x).size())
                x = torch.cat((x, clayer(outs[j])), dim=1)

            x = self.sv_blocks[i](x)
            outs.append(x)

        _crops = self.crop_layers[-1]
        for j, clayer in enumerate(_crops):
            # print(j)
            # print(x.size())
            # print(outs[j].size())
            # print(clayer)
            x = torch.cat((x, clayer(outs[j])), dim=1)

        x = nn.AvgPool2d(kernel_size=(x.size(-2), x.size(-1)))(x)
        x = nn.Flatten()(x)
#         print(x.size())
        x = self.ln(x)
        x = self.batchnorm(x)
        # x = self.relu(x)
        x = self.ln2(x)
        x = self.batchnorm2(x)

        return x


In [12]:
device = torch.device('cuda')
print(device)

model = SkippedVGG(in_size=(1,48,48), nb_blocks=4, n_classes=7, init_pf=7, device=device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.008)

# model.train()

# model = model.to(device)

cuda


In [94]:
model.sv_blocks

[SVblock(
   (conv1): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU()
   (maxpool): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
 ), SVblock(
   (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (batchnorm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU()
   (maxpool): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
 ), SVblock(
   (conv1): Conv2d(384, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (batchnorm): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=Tru

In [ ]:
# model(torch.rand(64,1,48,48).to(device))

In [13]:
model = model.to(device)

model_path = '/tf/data/Quan/fer2013/skippedvgg.pt'
best_acc = 0.0
hist = []

for epoch in range(200):  # loop over the dataset multiple times
    
    print('Epoch: ', epoch + 1)
    running_loss = 0.0
    running_acc = 0.0

    # TRAIN
    model.train()
    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        acc = float((torch.argmax(outputs, dim=1) == labels).float().sum()/labels.size(0))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        running_acc += acc
        print('\t - Step %d: loss: %.3f acc: %.3f' % (i+1, loss.item(), acc))

    print('- Avg.loss: %.3f  | Avg.acc: %.3f' % (running_loss / (i+1), running_acc / (i+1)))
    avgloss = running_loss / (i+1)
    avgacc = running_acc / (i+1)

    # EVALUATE
    model.eval()
    running_valloss = 0.0
    running_valacc = 0.0
    for i,data in enumerate(val_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        acc = float((torch.argmax(outputs, dim=1) == labels).float().sum()/labels.size(0))
        loss = criterion(outputs, labels)

        running_valloss += loss.item()
        running_valacc += acc

    print('- Avg. val_loss: %.3f  | Avg. val_acc: %.3f' % (running_valloss / (i+1), running_valacc / (i+1)))

    avgvalloss = running_valloss / (i+1)
    avgvalcc = running_valacc / (i+1)

    hist.append([avgloss, avgvalloss, avgacc, avgvalcc])
    if best_acc < (running_valacc / (i+1)):
        best_acc = (running_valacc / (i+1))
        torch.save(model, model_path)
        print('* Update optimal model')

print('Finished Training')

Epoch:  1
	 - Step 1: loss: 2.376 acc: 0.172
	 - Step 2: loss: 2.098 acc: 0.188
	 - Step 3: loss: 2.153 acc: 0.219
	 - Step 4: loss: 2.368 acc: 0.133
	 - Step 5: loss: 2.116 acc: 0.188
	 - Step 6: loss: 2.220 acc: 0.234
	 - Step 7: loss: 2.178 acc: 0.172
	 - Step 8: loss: 2.000 acc: 0.234
	 - Step 9: loss: 2.144 acc: 0.180
	 - Step 10: loss: 2.051 acc: 0.211
	 - Step 11: loss: 2.162 acc: 0.180
	 - Step 12: loss: 2.109 acc: 0.219
	 - Step 13: loss: 2.125 acc: 0.203
	 - Step 14: loss: 1.958 acc: 0.297
	 - Step 15: loss: 1.942 acc: 0.234
	 - Step 16: loss: 2.112 acc: 0.188
	 - Step 17: loss: 2.020 acc: 0.188
	 - Step 18: loss: 1.933 acc: 0.211
	 - Step 19: loss: 2.047 acc: 0.211
	 - Step 20: loss: 1.840 acc: 0.297
	 - Step 21: loss: 2.053 acc: 0.180
	 - Step 22: loss: 1.957 acc: 0.258
	 - Step 23: loss: 1.888 acc: 0.312
	 - Step 24: loss: 1.828 acc: 0.289
	 - Step 25: loss: 1.852 acc: 0.250
	 - Step 26: loss: 1.893 acc: 0.273
	 - Step 27: loss: 1.877 acc: 0.266
	 - Step 28: loss: 1.856 ac

- Avg. val_loss: 1.856  | Avg. val_acc: 0.306


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type SkippedVGG. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type SVblock. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type DownSampling. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


* Update optimal model
Epoch:  2
	 - Step 1: loss: 1.555 acc: 0.352
	 - Step 2: loss: 1.580 acc: 0.414
	 - Step 3: loss: 1.521 acc: 0.383
	 - Step 4: loss: 1.426 acc: 0.484
	 - Step 5: loss: 1.508 acc: 0.391
	 - Step 6: loss: 1.573 acc: 0.398
	 - Step 7: loss: 1.521 acc: 0.406
	 - Step 8: loss: 1.513 acc: 0.445
	 - Step 9: loss: 1.596 acc: 0.391
	 - Step 10: loss: 1.697 acc: 0.359
	 - Step 11: loss: 1.486 acc: 0.359
	 - Step 12: loss: 1.577 acc: 0.406
	 - Step 13: loss: 1.592 acc: 0.398
	 - Step 14: loss: 1.672 acc: 0.312
	 - Step 15: loss: 1.584 acc: 0.367
	 - Step 16: loss: 1.501 acc: 0.406
	 - Step 17: loss: 1.498 acc: 0.406
	 - Step 18: loss: 1.551 acc: 0.391
	 - Step 19: loss: 1.523 acc: 0.414
	 - Step 20: loss: 1.614 acc: 0.367
	 - Step 21: loss: 1.583 acc: 0.422
	 - Step 22: loss: 1.505 acc: 0.469
	 - Step 23: loss: 1.524 acc: 0.406
	 - Step 24: loss: 1.594 acc: 0.398
	 - Step 25: loss: 1.468 acc: 0.414
	 - Step 26: loss: 1.603 acc: 0.336
	 - Step 27: loss: 1.515 acc: 0.438
	 - 

	 - Step 225: loss: 1.362 acc: 0.432
- Avg.loss: 1.524  | Avg.acc: 0.412
- Avg. val_loss: 1.579  | Avg. val_acc: 0.396
* Update optimal model
Epoch:  3
	 - Step 1: loss: 1.517 acc: 0.492
	 - Step 2: loss: 1.481 acc: 0.438
	 - Step 3: loss: 1.358 acc: 0.523
	 - Step 4: loss: 1.279 acc: 0.570
	 - Step 5: loss: 1.426 acc: 0.477
	 - Step 6: loss: 1.433 acc: 0.461
	 - Step 7: loss: 1.510 acc: 0.453
	 - Step 8: loss: 1.553 acc: 0.367
	 - Step 9: loss: 1.381 acc: 0.477
	 - Step 10: loss: 1.557 acc: 0.391
	 - Step 11: loss: 1.317 acc: 0.484
	 - Step 12: loss: 1.382 acc: 0.523
	 - Step 13: loss: 1.460 acc: 0.477
	 - Step 14: loss: 1.372 acc: 0.484
	 - Step 15: loss: 1.610 acc: 0.406
	 - Step 16: loss: 1.385 acc: 0.461
	 - Step 17: loss: 1.324 acc: 0.492
	 - Step 18: loss: 1.581 acc: 0.375
	 - Step 19: loss: 1.466 acc: 0.406
	 - Step 20: loss: 1.459 acc: 0.461
	 - Step 21: loss: 1.529 acc: 0.445
	 - Step 22: loss: 1.466 acc: 0.406
	 - Step 23: loss: 1.329 acc: 0.523
	 - Step 24: loss: 1.423 acc:

...

	 - Step 195: loss: 1.363 acc: 0.469
	 - Step 196: loss: 1.218 acc: 0.609
	 - Step 197: loss: 1.343 acc: 0.508
	 - Step 198: loss: 1.342 acc: 0.500
	 - Step 199: loss: 1.240 acc: 0.523
	 - Step 200: loss: 1.327 acc: 0.477
	 - Step 201: loss: 1.228 acc: 0.477
	 - Step 202: loss: 1.304 acc: 0.500
	 - Step 203: loss: 1.428 acc: 0.398
	 - Step 204: loss: 1.250 acc: 0.555
	 - Step 205: loss: 1.272 acc: 0.523
	 - Step 206: loss: 1.400 acc: 0.453
	 - Step 207: loss: 1.242 acc: 0.523
	 - Step 208: loss: 1.291 acc: 0.547
	 - Step 209: loss: 1.405 acc: 0.516
	 - Step 210: loss: 1.279 acc: 0.508
	 - Step 211: loss: 1.174 acc: 0.570
	 - Step 212: loss: 1.192 acc: 0.562
	 - Step 213: loss: 1.212 acc: 0.547
	 - Step 214: loss: 1.434 acc: 0.422
	 - Step 215: loss: 1.119 acc: 0.609
	 - Step 216: loss: 1.262 acc: 0.531
	 - Step 217: loss: 1.459 acc: 0.414
	 - Step 218: loss: 1.325 acc: 0.539
	 - Step 219: loss: 1.386 acc: 0.484
	 - Step 220: loss: 1.222 acc: 0.578
	 - Step 221: loss: 1.360 acc: 0.492
	

	 - Step 193: loss: 1.284 acc: 0.492
	 - Step 194: loss: 1.212 acc: 0.594
	 - Step 195: loss: 1.541 acc: 0.391
	 - Step 196: loss: 1.196 acc: 0.578
	 - Step 197: loss: 1.408 acc: 0.469
	 - Step 198: loss: 1.339 acc: 0.469
	 - Step 199: loss: 1.368 acc: 0.516
	 - Step 200: loss: 1.306 acc: 0.508
	 - Step 201: loss: 1.194 acc: 0.523
	 - Step 202: loss: 1.324 acc: 0.461
	 - Step 203: loss: 1.157 acc: 0.562
	 - Step 204: loss: 1.311 acc: 0.523
	 - Step 205: loss: 1.225 acc: 0.578
	 - Step 206: loss: 1.331 acc: 0.531
	 - Step 207: loss: 1.182 acc: 0.523
	 - Step 208: loss: 1.281 acc: 0.500
	 - Step 209: loss: 1.337 acc: 0.469
	 - Step 210: loss: 1.332 acc: 0.516
	 - Step 211: loss: 1.345 acc: 0.469
	 - Step 212: loss: 1.572 acc: 0.398
	 - Step 213: loss: 1.378 acc: 0.430
	 - Step 214: loss: 1.374 acc: 0.453
	 - Step 215: loss: 1.179 acc: 0.508
	 - Step 216: loss: 1.182 acc: 0.562
	 - Step 217: loss: 1.292 acc: 0.547
	 - Step 218: loss: 1.298 acc: 0.492
	 - Step 219: loss: 1.287 acc: 0.461
	

	 - Step 191: loss: 1.378 acc: 0.469
	 - Step 192: loss: 1.139 acc: 0.594
	 - Step 193: loss: 1.240 acc: 0.547
	 - Step 194: loss: 1.388 acc: 0.445
	 - Step 195: loss: 1.343 acc: 0.516
	 - Step 196: loss: 1.354 acc: 0.484
	 - Step 197: loss: 1.359 acc: 0.500
	 - Step 198: loss: 1.347 acc: 0.508
	 - Step 199: loss: 1.280 acc: 0.484
	 - Step 200: loss: 1.302 acc: 0.531
	 - Step 201: loss: 1.230 acc: 0.508
	 - Step 202: loss: 1.357 acc: 0.516
	 - Step 203: loss: 1.164 acc: 0.555
	 - Step 204: loss: 1.243 acc: 0.516
	 - Step 205: loss: 1.269 acc: 0.469
	 - Step 206: loss: 1.238 acc: 0.531
	 - Step 207: loss: 1.348 acc: 0.508
	 - Step 208: loss: 1.320 acc: 0.531
	 - Step 209: loss: 1.356 acc: 0.484
	 - Step 210: loss: 1.280 acc: 0.477
	 - Step 211: loss: 1.284 acc: 0.523
	 - Step 212: loss: 1.217 acc: 0.523
	 - Step 213: loss: 1.273 acc: 0.477
	 - Step 214: loss: 1.351 acc: 0.492
	 - Step 215: loss: 1.442 acc: 0.383
	 - Step 216: loss: 1.223 acc: 0.562
	 - Step 217: loss: 1.365 acc: 0.484
	

	 - Step 189: loss: 1.314 acc: 0.508
	 - Step 190: loss: 1.217 acc: 0.578
	 - Step 191: loss: 1.331 acc: 0.508
	 - Step 192: loss: 1.248 acc: 0.547
	 - Step 193: loss: 1.319 acc: 0.484
	 - Step 194: loss: 1.284 acc: 0.539
	 - Step 195: loss: 1.338 acc: 0.523
	 - Step 196: loss: 1.223 acc: 0.602
	 - Step 197: loss: 1.171 acc: 0.555
	 - Step 198: loss: 1.306 acc: 0.508
	 - Step 199: loss: 1.179 acc: 0.523
	 - Step 200: loss: 1.307 acc: 0.516
	 - Step 201: loss: 1.213 acc: 0.523
	 - Step 202: loss: 1.349 acc: 0.508
	 - Step 203: loss: 1.241 acc: 0.531
	 - Step 204: loss: 1.282 acc: 0.523
	 - Step 205: loss: 1.361 acc: 0.461
	 - Step 206: loss: 1.421 acc: 0.500
	 - Step 207: loss: 1.345 acc: 0.430
	 - Step 208: loss: 1.348 acc: 0.500
	 - Step 209: loss: 1.254 acc: 0.547
	 - Step 210: loss: 1.144 acc: 0.602
	 - Step 211: loss: 1.200 acc: 0.578
	 - Step 212: loss: 1.391 acc: 0.422
	 - Step 213: loss: 1.343 acc: 0.531
	 - Step 214: loss: 1.246 acc: 0.484
	 - Step 215: loss: 1.257 acc: 0.539
	

KeyboardInterrupt: 